## Exercise 1: `mersenne_numbers`

A Mersenne number is any number that can be written as $2^p - 1$ for some $p$. For example, 3 is a Mersenne number ($2^2 - 1$) as is 31 ($2^5 - 1$). We will see later on that it is easy to test if Mersenne numbers are prime.

Write a function that accepts an exponent $p$ and returns the corresponding Mersenne number.

In [2]:
def mersenne_number(p):
    return ((2**p) -1)

Mersenne numbers can only be prime if their exponent, $p$, is prime. Make a list of the Mersenne numbers for all primes $p$ between 3 and 65 (there should be 17 of them).

Hint: It may be useful to modify the `is_prime` and `get_primes` functions from [the Program Flow notebook](https://www.codecademy.com/forum_questions/51f239449c4e9d4e3c001f43) for use in this problem.

In [12]:
def is_prime(number):
    if number <= 1:
        return False
    
    for factor in range(2, number):
        if number % factor == 0:
            return False

    return True


def print_primes(n):
    my_lst = []
    for number in range(3, n+1):
        if is_prime(number):
            my_lst.append(number)
    return my_lst

In [13]:
my_lst = print_primes(65)    #This prints the prime numbers between 3 and  65
print(my_lst)

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]


In [14]:
another_lst = []
for i in my_lst:
    another_lst.append(mersenne_number(i))
print(another_lst)

[7, 31, 127, 2047, 8191, 131071, 524287, 8388607, 536870911, 2147483647, 137438953471, 2199023255551, 8796093022207, 140737488355327, 9007199254740991, 576460752303423487, 2305843009213693951]


The next cell shows a dummy solution, a list of 17 sevens. Alter the next cell to make use of the functions you've defined above to create the appropriate list of Mersenne numbers.

In [15]:
mersennes = [mersenne_number(i) for i in my_lst]
print(mersennes)

[7, 31, 127, 2047, 8191, 131071, 524287, 8388607, 536870911, 2147483647, 137438953471, 2199023255551, 8796093022207, 140737488355327, 9007199254740991, 576460752303423487, 2305843009213693951]


## Exercise 2: `lucas_lehmer`

We can test if a Mersenne number is prime using the [Lucas-Lehmer test](https://en.wikipedia.org/wiki/Lucas%E2%80%93Lehmer_primality_test). First let's write a function that generates the sequence used in the test. Given a Mersenne number with exponent $p$, the sequence can be defined as

$$ n_0 = 4 $$
$$ n_i = (n_{i-1}^2 - 2) mod (2^p - 1) $$

Write a function that accepts the exponent $p$ of a Mersenne number and returns the Lucas-Lehmer sequence up to $i = p - 2$ (inclusive). Remember that the [modulo operation](https://en.wikipedia.org/wiki/Modulo_operation) is implemented in Python as `%`.

In [16]:
def lucas_lehmer(p):
    ll_seq = [4]
    m = 2**p - 1
    for i in range(1,p-1):
        n_i = ((ll_seq[i-1])** 2 - 2) % m
        ll_seq.append(n_i)
    return ll_seq

In [17]:
print(lucas_lehmer(17))

[4, 14, 194, 37634, 95799, 119121, 66179, 53645, 122218, 126220, 70490, 69559, 99585, 78221, 130559, 0]


## Exercise 3: `mersenne_primes`

For a given Mersenne number with exponent $p$, the number is prime if the Lucas-Lehmer series is 0 at position $p-2$. Write a function that tests if a Mersenne number with exponent $p$ is prime. Test if the Mersenne numbers with prime $p$ between 3 and 65 (i.e. 3, 5, 7, ..., 61) are prime. Your final answer should be a list of tuples consisting of `(Mersenne exponent, 0)` (or `1`) for each Mersenne number you test, where `0` and `1` are replacements for `False` and `True` respectively.

_HINT: The `zip` function is useful for combining two lists into a list of tuples_

In [18]:
def ll_prime(p):
    if lucas_lehmer(p)[-1] == 0:
        return 1
    return 0

In [19]:
list(zip((i for i in my_lst),(ll_prime(i) for i in my_lst)))

[(3, 1),
 (5, 1),
 (7, 1),
 (11, 0),
 (13, 1),
 (17, 1),
 (19, 1),
 (23, 0),
 (29, 0),
 (31, 1),
 (37, 0),
 (41, 0),
 (43, 0),
 (47, 0),
 (53, 0),
 (59, 0),
 (61, 1)]

## Exercise 4: Optimize `is_prime`

You might have noticed that the primality check `is_prime` we developed before is somewhat slow for large numbers. This is because we are doing a ton of extra work checking every possible factor of the tested number. We will use two optimizations to make a `is_prime_fast` function.

The first optimization takes advantage of the fact that two is the only even prime.  Thus we can check if a number is even and as long as its greater than 2, we know that it is not prime.

Our second optimization takes advantage of the fact that when checking factors, we only need to check odd factors up to the square root of a number.  Consider a number $n$ decomposed into factors $n=ab$.  There are two cases, either $n$ is prime and without loss of generality, $a=n, b=1$ or $n$ is not prime and $a,b \neq n,1$.  In this case, if $a > \sqrt{n}$, then $b<\sqrt{n}$.  So we only need to check all possible values of $b$ and we get the values of $a$ for free!  This means that even the simple method of checking factors will increase in complexity as a square root compared to the size of the number instead of linearly.

Lets write the function to do this and check the speed!  `is_prime_fast` will take a number and return whether or not it is prime.

You will see the functions followed by a cell with an `assert` statement.  These cells should run and produce no output, if they produce an error, then your function needs to be modified.  Do not modify the assert statements, they are exactly as they should be!

In [20]:
import math
def is_prime_fast(number):
    
    if number < 2 and number% 2 == 0:
        return False
    if number == 2:
        return True
    i = 2
    # This will loop from 2 to int(sqrt(x))
    while i*i <= number:
        # Check if i divides x without leaving a remainder
        if number % i == 0:
            # This means that n has a factor in between 2 and sqrt(n)
            # So it is not a prime number
            return False
        i += 1
    # If we did not find any factor in the above loop,
    # then n is a prime number
    return True  

Now lets check the timing, here we will use the `%%timeit` magic which will time the execution of a particular cell.

In [22]:
%%timeit
is_prime(67867967)

4.94 s ± 116 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
is_prime_fast(67867967)

1.2 ms ± 26.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Now return a function which will find all prime numbers up to and including $n$. Submit this function to the grader.

In [24]:
def get_primes_fast(n):
    return [number for number in range(2, n+1) if is_prime_fast(number)]

In [27]:
print(get_primes_fast(100)) #Prints all prime numbers between 1-100

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
